In [ ]:
## importing python modules needed
from scipy import interpolate
import numpy as np
import pandas as pd
import os as os
import csv
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline
import numpy.ma as ma

In [ ]:
os.chdir(r'C:\Users\Dell\Box\Arena_Lab_Data\Danish\PMA_Multilayer_Project\09202022 ML Measurements Co_Pt_Cu')
cwd = os.getcwd()
print (cwd)

In [ ]:
print (os.listdir(cwd))

In [ ]:
files='08152022_TA1_PT1.8X2_CO1.2X2_CU2_GLASS_OOP_00001.DAT'

PPMS_data = pd.read_csv(files, delimiter = ',', skiprows = 30)

In [ ]:
x= PPMS_data["Magnetic Field (Oe)"]
y= PPMS_data['Moment (emu)']

In [ ]:
xmax=int(PPMS_data["Magnetic Field (Oe)"].max())#MOKE data would be min field since start with neg 
print (xmax)


xlast=int(PPMS_data["Magnetic Field (Oe)"].iloc[-1])
print  (xlast)

xtol=np.arange(xmax*.95,xmax*1.05)

if xlast not in xtol:
    print ("NO GOOD YO")
    pass
else:
    print (xlast)

In [ ]:
#plt.title(file_title + " MH")
plt.plot(x, y)
plt.grid()
plt.xlabel("Field (Oe)")
plt.ylabel("Moment (emu)")
#pdf.savefig(bbox_inches = "tight")

In [ ]:
PPMS_data.idxmax

In [ ]:
#Hysteresis starts at 0 oe, goes to 5T, swings down to -5T and back to 5T. We don't care about the first 0-5T so we
#want to drop it from the array. There are 3 times it hits 5T; 0-5T, 5T- -5T, -5T - 5T. We will find those three 
#index locations and the smallest one will find the smallest index and that will correspond to the 0-5T run. 
max_locs = PPMS_data["Magnetic Field (Oe)"].sort_values(ascending=False)[0:1].index.tolist()
PPMS_data_red = PPMS_data[np.min(max_locs)+1:] #Keep only values after the 0-5T run

PPMS_data_red = PPMS_data_red.reset_index(drop = True)

len_red = np.int(len(PPMS_data_red)/2) #We want to fit the +/- 5T region so this corresponds to half the full loop.

x_axis = PPMS_data_red["Magnetic Field (Oe)"]
x_labs = "Magnetic Field (Oe)"

plt.show()

plt.plot(x_axis, PPMS_data_red["Moment (emu)"])
plt.xlabel(x_labs)
plt.ylabel("Moment (emu)")
plt.grid()

In [ ]:
#Now, we want to, again, background correct the edges of the diamagnetic corrected graph.
#Diamagnetic correction
mask_set = np.zeros(x_axis.shape)#Setup masking array same shape as array to be masked or similiar
mask_set[:(len_red)] = 1 #NP masked works by using 1 to mean mask this value so we add ones everywhere to 
mask_set[(len_red+50):] = 1
#our zero array where we want to mask.
fit_region = ma.masked_array(PPMS_data_red["Moment (emu)"], mask_set) #Apply masking portion
#Linear fit of pre and post-edge.
coeffs = np.ma.polyfit(x_axis, fit_region, 1) #Now that the middle region is masked, fit the linear endpoints.
dia_line1 = np.poly1d(coeffs) #Turn coefficients into a fit equation
coe1=coeffs[0]
print (coe1)
dia_corr1 = PPMS_data_red["Moment (emu)"] - coe1*x_axis
print (coeffs)
plt.plot(x_axis, dia_corr1)
plt.xlabel(x_labs)
plt.ylabel("Moment (emu)")
plt.grid()

In [ ]:
center_dia_corr1 = np.mean(dia_corr1)
print (center_dia_corr1)

In [ ]:
#shifting down w.r.t. zero along vertical axis

new_dia_corr1=dia_corr1-center_dia_corr1
plt.plot(x_axis, new_dia_corr1)
plt.xlabel(x_labs)
plt.ylabel("Moment (emu)")
plt.grid()

In [ ]:
pMs = MH_bck_corr[:int(half_loop*0.05)].mean() #Average first 5% of points.
nMs = MH_bck_corr[half_loop: (half_loop + int(half_loop*0.05)):].mean() #Average 5% of points at other endpoint.
Ms_center = np.mean([pMs, nMs])
ax[1,1].set_title("Finding vertical center")
ax[1,1].plot(x_axis, MH_bck_corr,label = 'MH')
ax[1,1].hlines(pMs, x_axis[0],x_axis[half_loop], colors = 'seagreen', linestyles = 'dashed', label = "+Ms")
ax[1,1].hlines(nMs, x_axis[0],x_axis[half_loop], colors = 'coral', linestyles = 'dashed', label = "-Ms")
ax[1,1].hlines(Ms_center, x_axis[0], x_axis[half_loop], colors = 'blueviolet', linestyles = 'dashed', label = "Hori. Center")
ax[1,1].grid()
ax[1,1].set_xlabel("Voltage (V)")
ax[1,1].set_ylabel("MOKE Signal")
ax[1,1].legend()
MH_vcent = MH_bck_corr - Ms_center #Center data vertically about moment = 0 emu
MH_vcent_max= np.max(MH_vcent)
    
MH_vcent_min=np.min(MH_vcent)
MH_vcent_avg= (MH_vcent_max-MH_vcent_min)/2
    #print (MH_vcent_max,MH_vcent_min,MH_vcent_avg)
    
fig2, ax2 = plt.subplots(1,2, figsize = (16,6))
ax2[0].set_title("vertically centered around y = 0")
ax2[0].plot(x_axis, MH_vcent)
ax2[0].hlines(0, x_axis[0], x_axis[half_loop],  colors = 'black', linestyles = 'dashed')
ax2[0].grid()
ax2[0].set_xlabel("Voltage (V)")
ax2[0].set_ylabel("MOKE Signal")
data = np.array([x_axis, MH_vcent])
data = data.T
PPMS_data_blue=pd.DataFrame(data, columns=['Voltage (V)', 'MOKE Signal'])

MHpn=PPMS_data_blue[:half_loop] # the first half +to-
MHpn = MHpn.reset_index(drop = True)
MHpn_inter=interp1d(y=MHpn['MOKE Signal'],x=MHpn['Voltage (V)'], kind='linear')
    
MHnp=PPMS_data_blue[half_loop:] # the second half -to+
MHnp = MHnp.reset_index(drop = True)
MHnp_inter=interp1d(y=MHnp['MOKE Signal'], x=MHnp['Voltage (V)'], kind='linear')
int_ax = np.linspace(-1.7, 1.7, 2000)
H_c_ind = np.absolute(MHnp_inter(int_ax)).argmin()
zeropoint=int_ax[H_c_ind]
    # zeropoint=MHnp_inter(0)
print ('zeropoint: ', zeropoint)
    
H_c_ind = np.absolute(MHpn_inter(int_ax)).argmin()
zeropoint2=int_ax[H_c_ind]
    # zeropoint=MHpn_inter(0)
print('zeropoint2: ', zeropoint2)
    

Deltazero=(zeropoint-zeropoint2)
print (Deltazero)
Coersive=abs(Deltazero)/2
print ('Coercive field: ', Coersive)

ax2[1].set_title("Finding Horizontal center")
ax2[1].plot(x_axis, MH_vcent)
ax2[1].vlines(zeropoint, MH_vcent[0],MH_vcent[half_loop], colors = 'green', linestyles = 'dashed', label = "+C")
ax2[1].vlines(zeropoint2, MH_vcent[0],MH_vcent[half_loop],colors = 'coral', linestyles = 'dashed', label = "-C")
ax2[1].vlines((zeropoint-Coersive), MH_vcent[0], MH_vcent[half_loop], colors = 'blueviolet', linestyles = 'dashed', label = "Hori. Center")
ax2[1].set_xlabel("Voltage (V)")
ax2[1].set_ylabel("MOKE Signal")
ax2[1].grid()       

MH_hcent=x_axis-(zeropoint-Coersive)


fig3, ax3 = plt.subplots(1,2, figsize = (18,6))
ax3[0].set_title("Horizontally centered around x = 0")
ax3[0].plot(MH_hcent, MH_vcent)
ax3[0].vlines(0, MH_vcent[0], MH_vcent[half_loop], colors = 'black', linestyles = 'dashed')
ax3[0].set_xlabel("Voltage (V)")
ax3[0].set_ylabel("MOKE Signal")
ax3[0].grid()
ax3[0].set_xlabel("Voltage (V)")
#pdf.savefig(bbox_inches = "tight")

In [ ]:
MAX=PPMS_data_red.loc[PPMS_data_red['Magnetic Field (Oe)'].idxmax()]
print (MAX)

In [ ]:
Saturation= MAX['Moment (emu)']
print (Saturation)

In [ ]:
MIN=PPMS_data_red.loc[PPMS_data_red['Magnetic Field (Oe)'].idxmin()]
SAT=MIN['Moment (emu)']
print (SAT)

In [ ]:
MS=(abs(Saturation -SAT))/2
print (MS)

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
##############Fromt MOKE Code
###field_to_vol_inter = interp1d(Calib["Magnetic Field (kG)"], Calib["Voltage (V)"])####

#first need to cut the PPMS len red into 2
#PPMS_data_red = PPMS_data[np.min(max_locs)+1:]
MHpm=PPMS_data_red[0:len_red]
MHpm = MHpm.reset_index(drop = True)

MHpm_inter=interp1d(MHpm['Moment (emu)'],MHpm['Magnetic Field (Oe)'])

In [ ]:
MHmp=PPMS_data_red[len_red:]
MHmp = MHmp.reset_index(drop = True)

MHmp_inter=interp1d(MHmp['Moment (emu)'],MHmp['Magnetic Field (Oe)'])

In [ ]:
zeropoint=MHmp_inter(0)
print (zeropoint)

In [ ]:
zeropoint2=MHpm_inter(0)
print (zeropoint2)

In [ ]:
Coersive=(abs(zeropoint-zeropoint2))/2
print (Coersive)

In [ ]:
MR_inter=interp1d(MHpm['Magnetic Field (Oe)'],MHpm['Moment (emu)'])

In [ ]:
MR_value=MR_inter(0)
print (MR_value)

In [ ]:
MR_ratio=MR_value/MS
print (MR_ratio)